In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pyneb as pn

## Chemico-Physical properties of nebulae: Te, Ne, X$^i$/H$^+$, X/H 

### Te and Ne

Let's create some atoms ;-)

In [ ]:
O3 = pn.Atom('O', 3, NLevels=5)
N2 = pn.Atom('N', 2, NLevels=5)
O2 = pn.Atom('O', 2, NLevels=5)
S2 = pn.Atom('S', 2, NLevels=5)

Configurations of the energy levels:

In [ ]:
f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2, figsize=(10,10))
N2.plotGrotrian(ax=ax1)
O2.plotGrotrian(ax=ax2)
O3.plotGrotrian(ax=ax3)
S2.plotGrotrian(ax=ax4)

Which line ratios are Te-diagnostics, Ne-diagnostics?
- [NII] 6584 / 6548
- [NII] 5755 / 6584
- [OIII] 88 / 52 $\mu$m
- [SII] 6730 / 6716
- [OII] 3729 / 3729
- [OII] 3727+ / 7325+
- [SII] 4076 / 4070

others

Let's generate some arrays containing values for electron temperatures and densities:

In [ ]:
tem = np.linspace(6000, 16000, 100)
tem1 = 8000
tem2 = 12000
den = np.logspace(1, 5, 100)
den1 = 1e2
den2 = 1e4

We plot the [OIII] 4363/5007 line ration as a function of Te, for different densities.

In [ ]:
rO3 = O3.getEmissivity(tem=tem, den=den1, wave=4363) / O3.getEmissivity(tem=tem, den=den1, wave=5007)
rO3_2 = O3.getEmissivity(tem=tem, den=den2, wave=4363) / O3.getEmissivity(tem=tem, den=den2, wave=5007)
f, ax = plt.subplots(figsize=(8,6))
ax.plot(tem, rO3, label=r'N$_e$ = 10$^2$ cm$^{-3}$')
ax.plot(tem, rO3_2, label=r'N$_e$ = 10$^4$ cm$^{-3}$')
ax.set_xlabel('T$_e$ [K]')
ax.set_ylabel('[OIII] 4363/5007')
ax.set_yscale('log')
ax.legend();

We plot the [SII] 6730/6716 line ration as a function of Ne, for different temperatures.

In [ ]:
rS2 = S2.getEmissivity(tem=tem1, den=den, wave=6730) / S2.getEmissivity(tem=tem1, den=den, wave=6716)
rS2_2 = S2.getEmissivity(tem=tem2, den=den, wave=6730) / S2.getEmissivity(tem=tem2, den=den, wave=6716)
f, ax = plt.subplots(figsize=(8,6))
ax.plot(den, rS2, label=r'T$_e$ = 8,000 K')
ax.plot(den, rS2_2, label=r'T$_e$ = 12,000 K')
ax.set_xlabel('N$_e$ [cm$^{-3}$]')
ax.set_ylabel('[SII] 6730/6716')
ax.set_xscale('log')
ax.legend();

For the density diagnostic line ratios, some characteristics can easily be obtained

In [ ]:
print(S2.getLowDensRatio(wave1=6730, wave2=6716))
print(S2.getHighDensRatio(wave1=6730, wave2=6716))
print(S2.getDensityRange(wave1=6730, wave2=6716, tem=1e4))

Let's have a llok at the variation of some line ratio when both Te and Ne change

In [ ]:
O3_EG = pn.EmisGrid('O', 3)

More options are available

In [ ]:
O3_EG = pn.EmisGrid(elem='O', spec=3, n_tem=110, n_den=90, 
                    tem_min=5000., tem_max=20000., den_min=10., 
                    den_max=1e8, restore_file=None, atomObj=None)

One can obtain the emissivity for a given line

In [ ]:
O3_5007 = O3_EG.getGrid(wave=5007)

In [ ]:
O3_5007.shape

Or for a line ratio, or any arithmetic combination of lines or level transition

In [ ]:
O3_Te = O3_EG.getGrid(to_eval = 'L(4363)/L(5007)')

In [ ]:
plt.imshow(O3_Te);

Plotting facility is included in the object

In [ ]:
O3_EG.plotImage(to_eval = 'L(4363)/L(5007)')

Even nicer with contourplots:

In [ ]:
f, ax = plt.subplots(figsize=(7,5))
O3_EG.plotContours(to_eval = 'L(4363)/L(5007)', ax=ax)
# One can save the result to hang it over the desk
f.savefig('OIII_diag.pdf')

This is very important plot. It shows where the line ratio is a diagnostic and where itis not.

In [ ]:
# Select a maximum of 25 diagnostics
diags = ['[ArIII] 5192/7136', 
         '[ArIII] 9.0m/21.8m',
         '[ArIV] 4740/4711',
         '[CIII] 1909/1907',
         '[ClIII] 5538/5518',
         '[NII] 121m/20.5m',
         '[NII] 5755/6584',
         '[NeIII] 15.6m/36.0m',
         '[NeIII] 3869/15.6m',
         '[NeV] 14.3m/24.2m',
         '[NeV] 2973/3370+',
         '[OI] 5577/6300+',
         '[OI] 63m/147m',
         '[OII] 3726/3729',
         '[OII] 3727+/7325+',
         '[OIII] 1664+/5007',
         '[OIII] 5007/88m', 
         '[OIII] 4363/5007',
         '[OIV] 1400+/25.9m',
         '[OIV] 1401/1405',
         '[SII] 4072+/6720+',
         '[SII] 4069/4076', 
         '[SII] 6731/6716',
         '[SIII] 6312/9069',
         '[SIII] 18.7m/33.5m'
         ] 
# Build the 25 subplots
f, axes = plt.subplots(5, 5, figsize=(20,20))
# loop on the diagnostics
for i, d in enumerate(diags):
    # extract from the diagnostic dictionnary 
    # the expression to be evaluated
    to_eval = pn.diags_dict[d][1]
    atom = pn.diags_dict[d][0]
    # split e.g. obtain 'O', 3 from 'O3'
    elem, spec = pn.utils.misc.parseAtom(atom)
    # instantiate the EmisGrid object for the given ion 
    EG = pn.EmisGrid(elem, spec)
    # select the axis in which to plot
    ax = axes.ravel()[i]
    # make the plot
    EG.plotContours(to_eval = to_eval, ax=ax)
# Make the plot nicer
f.tight_layout()
# Save the result in a pdf file
f.savefig('Diagnostics.pdf')

## Observation object

We ccreate a file with observations and uncertainties. These latests are a kind of mean between

In [ ]:
%%writefile observations1.dat
LINE   MYOBJECT errMYOBJECT
S4_10.5m   7.00  0.25
Ne2_12.8m  8.30  0.25
Ne3_15.6m 34.10  0.20
S3_18.7m  10.00  0.20
O2_3726A  39.70  0.05
O2_3729A  18.60  0.05
Ne3_3869A 18.90  0.05
Ne3_3968A  6.4   0.10
S2_4069A   0.85  0.15
S2_4076A   0.45  0.15
O3_4363A   4.36  0.10
H1r_4861A 100.0  0.00
O3_5007A 435.09  0.05
N2_5755A   0.51  0.15
S3_6312A   0.76  0.15
O1_6300A   1.69  0.15
O1_6364A   0.54  0.15
N2_6548A   6.84  0.10
H1r_6563A 345.0  0.05
N2_6584A  19.00  0.10
S2_6716A   1.22  0.15
S2_6731A   2.18  0.15
Ar3_7136A  4.91  0.15
O2_7319A+  6.54  0.10
O2_7330A+  5.17  0.10

In [ ]:
pn.log_.level = 3
obs = pn.Observation('observations1.dat', fileFormat='lines_in_rows_err_cols', corrected=False) # fill obs with data read from smc_24.dat

In [ ]:
obs.printIntens(returnObs=True) # return_obs is used to print out the original observaions (redenned)

In [ ]:
obs.extinction.law = 'CCM89'  # define the extinction law from Cardelli et al.
obs.def_EBV(label1="H1r_6563A", label2="H1r_4861A", r_theo=2.85) # Compute theredenning correction
obs.correctData(normWave=4861)                      # Correct the data from attenuation

In [ ]:
print(obs.extinction.E_BV)
print(obs.extinction.cHbeta)

In [ ]:
obs.printIntens()

## Diagnostic diagrams

In [ ]:
pn.log_.level = 3
diags = pn.Diagnostics()
diags.addDiagsFromObs(obs)
for d in diags.diags:
    print(d, ':', diags.diags[d][1])

In [ ]:
# Clean the list of diagnostics from redundant line ratios
for d in ('[NII] 5755/6584', '[NII] 5755/6584+', '[NeIII] 3930+/15.6m'):
    try:
        del diags.diags[d]
    except:
        pass

Plot the diagnostic diagram

In [ ]:
pn.log_.level = 2
f, ax = plt.subplots(figsize=(10, 8))
emisgrids = pn.getEmisGridDict(atomDict=diags.atomDict)
diags.plot(emisgrids, obs, ax=ax)

## Determine Te and Ne

In [ ]:
O3.getTemDen(150., den=100., wave1=5007, wave2=4363, maxError=1.e-2)

In [ ]:
S2.getTemDen(1.57, tem=12500, to_eval="I(2,1) / I(3,1)")

## Determine Te and Ne at the same time

In [ ]:
t, d = diags.getCrossTemDen(diag_tem='[OIII] 4363/5007', diag_den='[SII] 6731/6716', obs=obs)

In [ ]:
print('Te = {:.0f} K, Ne = {:.0f} cm-3'.format(t,d))

In [ ]:
t2, d2 = diags.getCrossTemDen(diag_tem='[NII] 5755/6584', diag_den='[SII] 6731/6716', obs=obs)
print('Te = {:.0f} K, Ne = {:.0f} cm-3'.format(t2,d2))

In [ ]:
t3, d3 = diags.getCrossTemDen(diag_tem='[NeIII] 3869/15.6m', diag_den='[SII] 6731/6716', obs=obs)
print('Te = {:.0f} K, Ne = {:.0f} cm-3'.format(t3,d3))

The Te-Ne to be used to determine the ionic abundances depends on the IP of the corresponding element

In [ ]:
pn.print_IPs(N_elems=20, N_ions=8)

The getTemDen and getCrossTemDen are vectorized. Theye are even parallelized and to use multiplrocessor, one just have to do:

In [ ]:
pn.config.use_multiprocs() # pn.config.unuse_multiprocs()
print('Using {} procs.'.format(pn.config.Nprocs))

If there is a huge number of data to deal with, one can use an Artificial Neural Network. This is already implemented in PyNeb, but test and validation of the hyperparameters still need to be achieved.